# Result figures for *Enhanced spatio-temporal electric load forecasts with less data using active deep learning*

---

## Overview
1. Import and test results
2. Numerical results
3. Space and time selection
4. Budget vs. accuracy
5. Training and validation losses against unqueried candidates
6. Validation losses against all candidates
7. Query sequence importance
8. Exemplar predictions
9. Manuscript: Results summary
10. Manuscript: Data selection maps

In this notebook session, we summarize and visualize the results of our experiments. First test if all numerical results were computed with the same hyper parameters. Next, we plot our results and exemplar predictions so as to visually see and evaluate their meaning. Lastly, we create figures for our manuscript from experiments that best visualize our findings. We start with importing a number of packages that we use throughout this notebook session. 

## 10. Manuscript: Data selection maps

Our findings can have important implications for the energy transition to a carbon free power systems and therefore for mitigating climate change. The ADL method we propose can be used by distribution and transmission system operators for electricity to make overall more accurate predictions of electric load using budgets for installing smart meters and streaming their data more effectively. This figure is supposed to visualize how ADL can show us where to place sensors and when to collect their data as we proceed in time (a. - f.). We start with choosing a set of locations to place sensors uniformly at random (a.). Then, we collect training data for making spatio-temporal predictions and learn where to best place a relatively large set of new sensors next (b.). Next, we iteratively place fewer sensors to make better predictions in the most informative sequence (c. - f.).

In [11]:
%%capture 
# prevents figures being printed out if used at begining of cell


# define some hyper for plots
n_rows = 3
n_cols = 2
n_iter_plot = n_rows * n_cols - 1
n_meshes_surface = 10
FONTSIZE = 22

# import building meta data
path_to_building_meta = '../data/private/' + profile_set + '/meta/meta buildings.csv'
building_meta = pd.read_csv(path_to_building_meta)

def create_bottom_plot(df_initial_sensors):

    # get bound coordinates of all buildings
    min_lat = df_initial_sensors['building lat'].min()
    max_lat = df_initial_sensors['building lat'].max() 
    min_long = df_initial_sensors['building long'].min()
    max_long = df_initial_sensors['building long'].max()

    # create evenly sized arrays and meshed grid of lats and longs
    lat_surface = np.linspace(
        min_lat, 
        max_lat,
        num=n_meshes_surface
    )
    long_surface = np.linspace(
        min_long, 
        max_long,
        num=n_meshes_surface
    )
    long_surface, lat_surface  = np.meshgrid(long_surface, lat_surface)
    
    map_height = 0
    Z = np.full((len(lat_surface), 1), map_height)
    
    ax.scatter(
        df_initial_sensors['building long'], 
        df_initial_sensors['building lat'], 
        map_height,  
        alpha=1, marker='x', c='r', s=100
    ) 
    ax.plot_surface(
        long_surface, 
        lat_surface, 
        Z,  
        alpha=0.03
    )
    ax.set_zlim(map_height)
    
def create_plot(time_data, df, df_new_sensors):
    
    # get bound coordinates of all buildings
    min_lat = df['building lat'].min()
    max_lat = df['building lat'].max() 
    min_long = df['building long'].min()
    max_long = df['building long'].max()

    # create evenly sized arrays and meshed grid of lats and longs
    lat_surface = np.linspace(
        min_lat, 
        max_lat,
        num=n_meshes_surface
    )
    long_surface = np.linspace(
        min_long, 
        max_long,
        num=n_meshes_surface
    )
    long_surface, lat_surface  = np.meshgrid(long_surface, lat_surface)
    
    # update max time point
    min_time_point, max_time_point = min(time_data), max(time_data)
    map_height = max_time_point/3 * (1 + 3* iteration/n_iter_plot)
    shifting_time = max_time_point - map_height
    Z = np.full((len(lat_surface), 1), map_height)
    
    ax.scatter(
        df['building long'], 
        df['building lat'], 
        time_data - shifting_time, 
        c=time_data, alpha=0.7
    )
    ax.scatter(
        df_new_sensors['building long'], 
        df_new_sensors['building lat'], 
        map_height,  
        alpha=1, marker='x', c='r', s=100
    )
    ax.plot_surface(
        long_surface, 
        lat_surface, 
        Z,  
        alpha=0.03
    )
    ax.set_zlim(min_time_point - shifting_time, max_time_point)
    

def customize_plot(iteration):
    
    # set angle
    ax.view_init(30, 110)
    
    # Get rid of the panes
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

    # Get rid of the ticks
    ax.set_xticks([]) 
    ax.set_yticks([]) 
    ax.set_zticks([])

    # Add the labels
    ax.set_xlabel('long', fontsize=FONTSIZE)
    ax.set_ylabel('lat', fontsize=FONTSIZE)
    ax.set_zlabel('time', fontsize=FONTSIZE)

    # shift time (z) axis
    tmp_planes = ax.zaxis._PLANES 
    ax.zaxis._PLANES = ( 
        tmp_planes[2], tmp_planes[3], 
        tmp_planes[0], tmp_planes[1], 
        tmp_planes[4], tmp_planes[5]
    )
    
    # shift lat (y) axis
    ax.yaxis._PLANES = ( 
        tmp_planes[2], tmp_planes[3], 
        tmp_planes[0], tmp_planes[1], 
        tmp_planes[4], tmp_planes[5]
    )
    
    
    # set subplot titles
    ax.set_title(
        fig_title_list[plot_counter-1].format(iteration+1), 
        fontsize=FONTSIZE + 4
    )


# create a counter over the list of numeric results
result_index_counter = 0

# iterate over all considered prediction types
for index_pred, pred_type in enumerate(PRED_TYPE_LIST):
    
    # iterate over all considered parameter constellations
    for index_param, parameter in enumerate(PARAMETER_LIST):
        
        # get results df corresponding to currently iterated parameter and pred_type
        spacetime_result = spacetime_selection_list[result_index_counter]

        # increment result index counter
        result_index_counter += 1
        
        for AL_variable in AL_VARIABLES:
            
            for AL_variant in AL_VARIANTS:
        
                # create figure
                fig = plt.figure(figsize=(16, n_rows * 8))

                # set the fontsize for figures
                mpl.rcParams.update({'font.size': 16}) #FONTSIZE

                colname_initial_sensors = '{} - initial sensors'.format(pred_type)
                intial_sensors = spacetime_result[colname_initial_sensors]
                df_initial_sensors = pd.DataFrame({'building ID':intial_sensors})
                df_initial_sensors = df_initial_sensors.merge(building_meta, on='building ID', how='left')
                
                plot_counter= 1
                ax = fig.add_subplot(n_rows, n_cols, plot_counter, projection='3d')
                create_bottom_plot(df_initial_sensors)
                customize_plot(iteration=0)
                plot_counter += 1
                old_senors_AL_set = set(intial_sensors)
                old_senors_PL_set = set(intial_sensors)
                for iteration in range(n_iter_plot):

                    # create column names
                    colname_AL_times = '{} {} {} - iter {} time'.format(pred_type, AL_variable, AL_variant, iteration) 
                    colname_AL_spaces = '{} {} {} - iter {} space'.format(pred_type, AL_variable, AL_variant, iteration) 
                    
                    ### Plot AL results on left column ###
                    
                    # get data
                    space_data_AL = spacetime_result[colname_AL_spaces]
                    time_data_AL = spacetime_result[colname_AL_times]

                    # get new sensors and set old sensors
                    new_sensors_AL_set = set(space_data_AL).union(old_senors_AL_set)
                    new_sensors_AL_list = list(new_sensors_AL_set - old_senors_AL_set)
                    old_senors_AL_set = new_sensors_AL_set
                    
                    # assign lat and long to building IDs
                    df_AL = pd.DataFrame({'building ID':space_data_AL})
                    df_AL = df_AL.merge(building_meta, on='building ID', how='left')

                    df_new_sensors_AL = pd.DataFrame({'building ID':new_sensors_AL_list})
                    df_new_sensors_AL = df_new_sensors_AL.merge(building_meta, on='building ID', how='left')

                    # AL temporal scatter plot
                    ax = fig.add_subplot(n_rows, n_cols, plot_counter, projection='3d')
                    create_plot(time_data_AL, df_AL, df_new_sensors_AL)
                    customize_plot(iteration)
                    plot_counter += 1

                # create saving paths 
                saving_path = (
                    path_to_ADL_selection 
                    + pred_type
                    + ' '
                    + parameter
                    + ' '
                    + AL_variable
                    + ' '
                    + AL_variant
                    + '.pdf'
                )

                # create a legend
                legend_elements = [
                    Line2D([0], [0], marker='o', color='w', markerfacecolor='b', markersize=10, label='query in space-time'),
                    Line2D([0], [0], marker='X', color='w', markerfacecolor='r', markersize=15, label='new sensor in space')
                ]

                # set layout tight
                fig.tight_layout()

                fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.6, 1), fontsize=FONTSIZE)

                # save figures
                fig.savefig(saving_path, bbox_inches="tight")